In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import os

In [18]:
# --- Configuration ---
DB_CONFIG = {
    "host": "localhost",
    "database": "bank_reviews", # Use the database you just created
    "user": "postgres",     
    "password": "password" 
}

# Define file paths (Ensure this path correctly points to your analysis output)
PROJECT_ROOT = 'C:/Users/USER/Desktop/Projects/Customer Experience Analytics/Customer-Experience-Analytics' 
ANALYSIS_DATA_FILE = os.path.join(PROJECT_ROOT, 'data', 'processed', 'analyzed_bank_reviews.csv')

In [30]:
# --- ETL Functions ---

def load_data_to_db():
    """Manages the connection, data loading, and insertion into PostgreSQL."""
    try:
        # Load the analyzed CSV file
        df = pd.read_csv(ANALYSIS_DATA_FILE)
        print(f"Loaded {len(df)} reviews from CSV.")

        # Converts date strings to Python date objects, coercing invalid/missing data to None
        df['date'] = pd.to_datetime(
            df['date'],  # <--- ACCESSING THE 'date' COLUMN
            errors='coerce' 
        ).dt.date

    except FileNotFoundError:
        print(f"!!! ERROR: Analysis data not found at {ANALYSIS_DATA_FILE}")
        return 

    conn = None
    try:
        # 1. Connect to the database
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()

        # 2. Insert Banks (Unique bank names first)
        bank_map = {}
        # Ensure 'bank' and 'source' columns exist from your CSV
        unique_banks = df[['bank', 'source']].drop_duplicates().itertuples(index=False)
        print("\n--- Inserting Unique Banks ---")
        
        for bank_name, app_name in unique_banks:
            # Insert the bank and use ON CONFLICT DO NOTHING for reruns
            insert_bank_query = sql.SQL("""
                INSERT INTO Banks (bank_name, app_name) 
                VALUES (%s, %s)
                ON CONFLICT (bank_name) DO NOTHING;
            """)
            cur.execute(insert_bank_query, (bank_name, app_name))
            
            # Retrieve the bank_id (newly created or existing)
            fetch_id_query = sql.SQL("SELECT bank_id FROM Banks WHERE bank_name = %s;")
            cur.execute(fetch_id_query, (bank_name,))
            bank_id = cur.fetchone()[0]
            bank_map[bank_name] = bank_id


        ## 3. Insert Reviews
        print("\n--- Inserting Review Data ---")
        insert_review_query = sql.SQL("""
            INSERT INTO Reviews (
                bank_id, review_text, rating, review_date, sentiment_label,  
                sentiment_score, identified_theme, source
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """)

        
        # Prepare data for insertion
        inserted_count = 0
        for index, row in df.iterrows():
            bank_id = bank_map[row['bank']]
            
            # Handle potential NaN values for safe insertion
            review_text = row['review_text'] if pd.notna(row['review_text']) else None
            
            # --- CORRECTION 2: ACCESSING THE 'date' COLUMN ---
            date_value = row['date'] if pd.notna(row['date']) else None 
            
            cur.execute(insert_review_query, (
                bank_id,
                review_text,
                row['rating'],
                date_value, # <--- PASSING THE CLEANED 'date_value'
                row['sentiment_label'],
                row['sentiment_score'],
                row['identified_theme'],
                row['source']
            ))
            inserted_count += 1
            
        # Commit the transaction to make changes permanent
        conn.commit()
        print(f"\nSuccessfully inserted {inserted_count} review entries.")
        print("Task 3 Insertion KPI met.")

    except Exception as error:
        print(f"Database error occurred: {error}")
        if conn:
            conn.rollback() 
            
    finally:
        if conn:
            cur.close()
            conn.close()
            print("Database connection closed.")

if __name__ == "__main__":
    load_data_to_db()

Loaded 1500 reviews from CSV.

--- Inserting Unique Banks ---

--- Inserting Review Data ---

Successfully inserted 1500 review entries.
Task 3 Insertion KPI met.
Database connection closed.
